# Template Prototype

In [1]:
# Imports
import pandas as pd
from binance.client import Client
from binance.enums import * #https://github.com/sammchardy/python-binance/blob/master/binance/enums.py
import datetime
import json
import math

import schedule
import time
from datetime import datetime, timedelta

import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from IPython.display import clear_output

def get_api_keys(site: str, api_type: str)->str:
    """
    gets api keys stored in api-keys/api-keys.txt
    site: 'binance'
    api_type: 'api', 'secret'
    """
    with open('../api-keys/api-keys.txt') as json_file:
        return json.load(json_file)[site][api_type]
# Constants
api_key = get_api_keys("binance", "api")
api_secret = get_api_keys("binance", "secret")

client = Client(api_key=api_key, api_secret=api_secret)

mode = 2

In [2]:
## Constants
ASSET_A = "CELR"
ASSET_B = "FET"
BASE = "USDT"
LOOKBACK = 1800
THRES = 0.6
SELL_THRES = 0.8

In [3]:
def get_margin_asset(asset:str, isolated=True):
    """No USDT. returns a dictionary with:
    - asset name
    - free
    - locked
    - borrowed
    - interest
    - netAsset"""
    if isolated:
        return get_isolated_margin_account(asset)["baseAsset"]
    else:
        return list(filter(lambda x: x['asset'] == asset, client.get_margin_account()["userAssets"]))[0]

def get_isolated_margin_account(base_asset: str):
    """Returns dict for isolated margin account for base_asset. Enter base_asset as 'FET'. Do NOT include USDT"""
    c = client.get_isolated_margin_account()
    return list(filter(lambda x: x["baseAsset"]["asset"] == base_asset, c["assets"]))[0]

def binance_ceil(x:float, dp:float):
    """returns the ceil to dp decimal places (to payback borrowed amounts). Includes 0.1% trading fee"""
    return math.ceil(x*1.001*(10 ** dp))/(10 ** dp)

def binance_floor(x:float, dp:float):
    """returns the floor to dp decimal places amount not including trading fee. (for liquidating)"""
    return math.floor(x*(10 ** dp))/(10 ** dp)

def go_long_short(long: str, l_amt: float, short: str, s_amt: float, base=BASE):
    """goes long and short. Enter long/short as 'XMR' rather than 'XMRUSDT'. """
    l_pair = long + base
    s_pair = short + base
    transaction = client.create_margin_loan(asset=short, amount=str(s_amt), isIsolated='TRUE', symbol=s_pair)
#     time.sleep(1)
    print("starting short order")
    order_s = client.create_margin_order(
        symbol=s_pair,
        side=SIDE_SELL,
        type=ORDER_TYPE_MARKET,
        quantity=s_amt,
        newOrderRespType = "FULL",
        isIsolated='TRUE')
    print(order_s)
    print("starting buy order")
    order_l = client.create_margin_order(
        symbol=l_pair,
        side=SIDE_BUY,
        type=ORDER_TYPE_MARKET,
        quantity=l_amt,
        newOrderRespType = "FULL",
        isIsolated='TRUE')
    print(order_l)
    send_message("ls", long=long, l_amt=str(l_amt), short=short, s_amt=str(s_amt), o_s=order_s, o_l=order_l)
    
def liquidate(long: str, short: str, base=BASE):
    """liquidates long and short position. Enter long as currently longing which asset"""
    l_pair = long + base
    s_pair = short + base
    l_amt = binance_floor(float(get_margin_asset(long)["free"]), 1)
    s_amt = binance_ceil(abs(float(get_margin_asset(short)["netAsset"])), 1)
    
    order = client.create_margin_order(
        symbol=l_pair,
        side=SIDE_SELL,
        type=ORDER_TYPE_MARKET,
        quantity=l_amt,
        newOrderRespType = "FULL",
        isIsolated='TRUE')
    order1 = client.create_margin_order(
        symbol=s_pair,
        side=SIDE_BUY,
        type=ORDER_TYPE_MARKET,
        quantity=s_amt,
        newOrderRespType = "FULL",
        isIsolated='TRUE')
    rp = str(abs(float(get_margin_asset(short)["free"])))
    transaction1 = client.repay_margin_loan(asset=short, amount=rp, isIsolated='TRUE', symbol=s_pair)
    rebalance_accounts(ASSET_A, ASSET_B)
    send_message("lq", o_s=order, o_b=order1)
    
def rebalance_accounts(a: str, b: str, base=BASE):
    """Rebalance isolated margin accounts a and b. Enter as 'FET' """
    a_usdt = float(get_isolated_margin_account(a)["quoteAsset"]["free"])
    b_usdt = float(get_isolated_margin_account(b)["quoteAsset"]["free"])
    transfer = str(round(abs(a_usdt - (a_usdt + b_usdt)/2), 6))
    from_account = ""
    to_account = ""
    if a_usdt > b_usdt:
        from_account = a + base
        to_account = b + base
    else:
        from_account = b + base
        to_account = a + base
    client.transfer_isolated_margin_to_spot(asset='USDT', symbol=from_account, amount=transfer)
    client.transfer_spot_to_isolated_margin(asset='USDT', symbol=to_account, amount=transfer)
    
def get_price(symbol:str):
    """returns the price. symbol MUST include USDT, ie ZECUSDT"""
    return float(client.get_recent_trades(symbol=symbol, limit=1)[0]["price"])

def trade_amt(symbol:str, total:float, base=BASE):
    """returns the amount of symbol to trade to be half of total. DONT include USDT"""
    p = get_price(symbol + BASE)
    return binance_floor(total/p, 1)

def get_long():
    """Returns None if long nothing, else ASSET_A or ASSET_B"""
    if float(get_margin_asset(ASSET_A)["borrowed"])>0: #Borrowed ASSET_A (ie long ASSET_B)
        return ASSET_B
    elif float(get_margin_asset(ASSET_B)["borrowed"])>0: #Borrwed XMR (ie long ZEC)
        return ASSET_A
    else:
        return None

In [4]:
def printer():
    """main printer. Fetches current position, fetches data, and performs buys/sells if necessary"""
    long = get_long() # Returns None if long nothing, else ASSET_A or ASSET_B

    # Get z_score
    z = get_z_score()
    while math.isnan(z): #z is nan, retry:
        time.sleep(1)
        z = get_z_score()
    
    # Get trading amount (won't be used if liquidating or NOT buying)
    # NOTE: ASSUMES isolated margin account for both A and B have the same amount of USDT
    total = float(get_isolated_margin_account(ASSET_A)["quoteAsset"]["free"])*0.95
    asset_a = trade_amt(ASSET_A, total)
    asset_b = trade_amt(ASSET_B, total)
    
    if z>THRES and long==None: #buy/sell #LONG A
        go_long_short(long=ASSET_A, l_amt=asset_a, short=ASSET_B, s_amt=asset_b)
    elif z<-THRES and long==None: #buy/sell #LONG B
        go_long_short(long=ASSET_B, l_amt=asset_b, short=ASSET_A, s_amt=asset_a)
    elif z>SELL_THRES and long==ASSET_B: #liquidate #LONG B
        liquidate(long=ASSET_B, short=ASSET_A)
    elif z<-SELL_THRES and long==ASSET_A: #liquidate #LONG A
        liquidate(long=ASSET_A, short=ASSET_B)
    
    send_message("z", z=str(z))
    change_mode(z, long)
    reset_client()
    
    return schedule.CancelJob

In [5]:
def change_mode(z:float, long):
    """Change the frequency of the checking"""
    global mode
    if long==None:
        mode = 2
    elif long==ASSET_A:
        if z > -SELL_THRES + 1.5: 
            mode = 10
        elif z > -SELL_THRES + 1.: 
            mode = 5
        elif z > -SELL_THRES + 0.5: 
            mode = 3
        else:
            mode = 2
    elif long==ASSET_B:
        if z < SELL_THRES - 1.5:
            mode = 10
        elif z < SELL_THRES - 1.:
            mode = 5
        elif z < SELL_THRES - .5:
            mode = 3
        else:
            mode = 2
    
def reset_client():
    """resets the client to prevent 'read operation timed out'"""
    global client
    global api_key
    global api_secret
    client = Client(api_key=api_key, api_secret=api_secret)

def get_z_score(): 
    '''gets the latest z-score, given hedge ratio hr.
    Warning, sometimes it gives nan, just rerun (binance's fault)'''
    a = get_minutely_data(ASSET_A + BASE)
    b = get_minutely_data(ASSET_B + BASE)
    a.set_index("timestamp", inplace=True)
    b.set_index("timestamp", inplace=True)
    
    df = pd.to_numeric(a.open.rename("A")).to_frame()
    df["B"] = pd.to_numeric(b.open)
    
    df.dropna(inplace=True)
    
    results = sm.ols(formula="B ~ A", data=df[['B', 'A']]).fit()
    hr = results.params[1]
    spread = pd.Series((df['B'] - hr * df['A'])).rename("spread").to_frame()
    spread["mean"] = spread.spread.rolling(LOOKBACK).mean()
    spread["std"] =  spread.spread.rolling(LOOKBACK).std()
    spread["zscore"] = pd.Series((spread["spread"]-spread["mean"])/spread["std"])
    
    return spread.iloc[-1].zscore

In [6]:
def get_filtered_dataframe(df):
    """filters columns and converts columsn to floats and ints respectively"""
    df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
    df = df.astype(np.float64)
    df["timestamp"] = df.timestamp.astype(np.int64)
    return df

def get_minutely_data(symbol:str, days=0.5):
    """smart gets minutely data. Enter symbol with USDT"""
    data_past = pd.read_csv(f"../data/{symbol}-past.csv")

    d = datetime.today() - timedelta(days=days)
    start_date = d.strftime("%d %b %Y %H:%M:%S")
    today = datetime.today().strftime("%d %b %Y %H:%M:%S")

    klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1MINUTE, start_date, today, 1000)
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data = get_filtered_dataframe(data)

    index = data_past.index[(data_past['timestamp'] == data.iloc[0].timestamp)].tolist()[0]
    data = pd.concat([data_past[:index], data], ignore_index=True, sort=False)

    klines = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1MINUTE)
    data_latest = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data_latest = get_filtered_dataframe(data_latest)

    index = data.index[(data['timestamp'] == data_latest.iloc[0].timestamp)].tolist()[0]
    result = pd.concat([data[:index], data_latest], ignore_index=True, sort=False)
    result.to_csv(f"../data/{symbol}-past.csv", index=False)
    return result

In [7]:
# Discord # Saving message
def get_message():
    """gets the message"""
    with open('message.txt') as json_file:
        new_message = json.load(json_file)
        return new_message
def send_message(m:str,  **k):
    data = get_message()
    if m == "ls":
        data["message"] = f"Went long {k['l_amt']} {k['long']} at ${k['o_l']['fills'][0]['price']} \nShort {k['s_amt']} {k['short']} at ${k['o_s']['fills'][0]['price']}"
        data["o_l"] = k['o_l']
        data["o_s"] = k['o_s']
    elif m == "lq":
        o_s = k['o_s']
        o_b = k['o_b']
        data["message"] = f"Liquidated! Sold {o_s['executedQty']} {o_s['symbol']} for {o_s['fills'][0]['price']} \nBought {o_b['executedQty']} {o_b['symbol']} for {o_b['fills'][0]['price']}"
        data["o_l"] = o_s
        data["o_s"] = o_b
    elif m == "z":
        data["z"] = k["z"]
    elif m == "mode":
        data["mode"] = k["mode"]
    with open('message.txt', 'w') as outfile:
        json.dump(data, outfile)

In [8]:
schedule.clear()
schedule.every().minute.at(":01").do(printer)

Every 1 minute at 00:00:01 do printer() (last run: [never], next run: 2021-07-18 12:57:01)

In [9]:
sleep = 1
mode = 2
while True:
    clear_output()
    print(f"Starting while loop: mode is {mode}")
    try:
        if schedule.get_jobs() != []:
            print(f"Run schedule")
            schedule.run_pending()
        else:
            print(f"Schedule is empty, setting it to run every {mode} minutes")
            schedule.every(mode).minutes.at(":01").do(printer)
            send_message("mode", mode=str(mode))
    except:
        print("An error occured, sleeping for 2 minutes")
        rest = 120
        schedule.clear()
        while True:
            time.sleep(rest)
            try: 
                reset_client()
                schedule.every().minute.at(":01").do(printer)
                break
            except:
                print(f"Error again, sleeping an additional 10 seconds, to {rest + 10} seconds")
                rest += 10
    print(f"sleeping for {sleep} seconds")
    time.sleep(1)

Starting while loop: mode is 10
Run schedule
sleeping for 1 seconds


KeyboardInterrupt: 